# Advanced Pandas techniques to boost performance!

https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6

Pycon Presentation - https://www.youtube.com/watch?v=HN5d490_KKk

We'll test and compare different methods of applying functions to pandas dataframes.
The function we'll be testing is the Haversine Distance formular which takes the cordinates of two points (lattitude and longitude) and calculates the straight line distance

In [0]:
import numpy as np

# Define a basic Haversine distance formula
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

## Remember, everytime you loop in Pandas a kitten dies.
Not really, but it's you who'll be suffering and I'll show you why

# Let's loop though a dataframe of cordinates and return a series with these distences

In [0]:
# Define a function to manually loop over all rows and return a series of distances
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [3]:
import pandas as pd

file_name = "https://raw.githubusercontent.com/rajeevratan84/datascienceforbusiness/master/new_york_hotels.csv"
df = pd.read_csv(file_name, encoding = "ISO-8859-1")
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


In [4]:
df.shape

(1631, 11)

In [5]:
# This is a built in magic ipython command that provides timing for code executed in the cell
%%timeit

# Run the haversine looping function
df['distance'] = haversine_looping(df)

1 loop, best of 3: 627 ms per loop


## Looping with iterrows()

In [6]:
%%timeit

# Haversine applied on rows via iteration
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, row['latitude'], row['longitude']))
df['distance'] = haversine_series

1 loop, best of 3: 219 ms per loop


## We got a 300% improvement! But can we get it any faster?

In [0]:
%%timeit

# Timing apply on the Haversine function
df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']), axis=1)

10 loops, best of 3: 60.3 ms per loop


### We went from 600ms to 60ms, a ten fold increase in speed!

In [0]:
 !pip install line_profiler
 %load_ext line_profiler

# Haversine applied on rows with line profiler
%lprun -f haversine df.apply(lambda row: haversine(40.671, -73.985, row['latitude'], row['longitude']), axis=1)


     |████████████████████████████████| 92kB 3.8MB/s 
  Created wheel for line-profiler: filename=line_profiler-2.1.2-cp36-cp36m-linux_x86_64.whl size=180891 sha256=71f8e313b0f5f09a7cacb58a38803e7564f6a17f009eaf7bea41188e9ece2581
  Stored in directory: /root/.cache/pip/wheels/05/7d/9b/aafbe8d78dc2b2c644d2efd2f060ab3258143860142575193a
Successfully built line-profiler


### We can note two things from the output.
1. Line 9 our trigonometric calcuation accounts for the bulk of our processing time, understandably so.
2. Each line is being hit 1631 times, that is because we're still **looping** so we iterate through every row in our dataframe

# Now for Vectorization!

- We've only been passing scalars to our Haversine function. Fortunately, all of the functions being used within the Haversine function, are also able to operate on arrays. 
- This makes the process of vectorizing our distance function quite simple as we can pass the entire series (columns) to it, performing all the calculations on the entire array simultaneously.

![alt text](https://github.com/rajeevratan84/datascienceforbusiness/blob/master/vectorization.png?raw=true)

In [0]:
%%timeit 

# Vectorized implementation of Haversine applied on Pandas series
df['distance'] = haversine(40.671, -73.985, df['latitude'], df['longitude'])

The slowest run took 14.31 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 2.52 ms per loop


In [0]:
# Run our line profiler to inspect further
%lprun -f haversine df['distance'] = haversine(40.671, -73.985, df['latitude'], df['longitude'])

We can see that each line is hit once! exactly what we wanted, however, line 9 still accounts for the bulk of our proessing time. Can that be sped up? 

## Let's use numpy vetorization instead of pandas

In [0]:
%%timeit

# Vectorized implementation of Haversine applied on NumPy arrays (note we use .values to access the numpy series)
df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

The slowest run took 8.09 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 274 µs per loop


# So we went from 609 ms using regular for loops, to 275 µs (or 0.275ms)!!! A 2000 fold increase in performance!